In [31]:
open System

In [43]:
///**Description**
/// Definition einer 
///
type Function<'a> =
    {
        Name: string;
        Evaluate: ('a->'a);
    }

///**Description**
/// Definition der einzelnen Teile welche einen Ausdruck ausmachen können.
///
type Expr =  
    | Con of int
    | Var of string
    | Add of Expr * Expr
    | Sub of Expr * Expr
    | Mult of Expr * Expr
    | Div of Expr * Expr
    | Power of Expr * Expr
    | Neg of Expr
    | Func of Function<double> * Expr
    
///**Description**
/// Überladung der Operatoren um direkt Expression-Instanzen zu Erzeugen.
///
type Expr with
    static member (+) (x, y) = Add(x, y)
    static member (+) (x, y) = Add(x, Con y)
    static member (+) (x, y) = Add(Con x, y)
    static member (-) (x, y) = Sub(x, y)
    static member (-) (x, y) = Sub(x, Con y)
    static member (-) (x, y) = Sub(Con x, y)
    static member (*) (x, y) = Mult(x, y)
    static member (*) (x, y) = Mult(x, Con y)
    static member (*) (x, y) = Mult(Con x, y)
    static member (/) (x, y) = Div(x, y)
    static member (/) (x, y) = Div(x, Con y)
    static member (/) (x, y) = Div(Con x, y)
    static member Pow (x, y) = Power(x, y)
    static member Pow (x, y) = Power(x, Con y)
    static member Pow (x, y) = Power(Con x, y)
    static member (~-) x     = Neg x
    static member (~+) x     = x    

In [58]:
///**Description**
/// Ableiten von Funktionen.
/// (aus organisatorischen Gründen ausgelagert)
///
let deriveFunc (f: Function<double>) (argument: Expr) : Expr =
    match f.Name with
    | "sin" ->
        (Func ({Name="cos"; Evaluate=Math.Cos}, argument))
    | _ ->
        failwith <| sprintf "Unknown function: %s" f.Name


let rec derive var expr =  
    let d = derive var
    match expr with
    | Var var -> Con 1                             // Identity Rule
    | Con x -> Con 0                               // Constant Rule
    | Func (f, argument) -> 
        Mult (
                (d argument),
                (deriveFunc f argument)
             )
    | Mult (Con x, y) | Mult (y, Con x) -> Con x   // Constant Factor Rule
    | Add (x, y) -> d x + d y                      // Sum Rule
    | Sub (x, y) -> d x - d y                      // Difference Rule
    | Mult (x, y) -> d x * y + x * d y             // Product Rule
    | Div (x, y) -> (d x * y - x * d y) / y ** 2   // Quotient Rule
    | Power (var, Con x) -> x * var ** (x - 1)     // Elementary Power Rule
    | rest -> failwith <| sprintf "failed to match: '%A'" rest

In [59]:
let x = Var "x"
let y = Var "y"
let z = Var "z"

In [60]:
let rec print expr =  
     match expr with
     | Add (x, y) -> sprintf "(%s + %s)" (print x) (print y)
     | Sub (x, y) -> sprintf "(%s - %s)" (print x) (print y)
     | Mult (x, y) -> sprintf "(%s * %s)" (print x) (print y)
     | Div (x, y) -> sprintf "(%s / %s)" (print x) (print y)
     | Power (x, y) -> sprintf "(%s ** %s)" (print x) (print y)
     | Func (x, y) -> sprintf "%s(%s)" (x.Name) (print y)
     | Neg x -> sprintf "-(%s)" (print x)
     | Var x -> x
     | Con x -> string x

In [63]:
(x ** 2 * (Func ({Name="sin"; Evaluate=Math.Sin}, x))**2)

Mult
  (Power (Var "x",Con 2),Power (Func ({Name = "sin";
                                       Evaluate = <fun:it@1-4>;},Var "x"),Con 2))

In [64]:
derive x (x ** 2 * (Func ({Name="sin"; Evaluate=Math.Sin}, x))) |> print

"(((2 * (x ** 1)) * sin(x)) + ((x ** 2) * (1 * cos(x))))"